In [ ]:
import pandas as pd


def count_equivalent_rows(df1, df2):
    df1_normalized = df1.copy()
    df2_normalized = df2.copy()

    df1_normalized[["response_a", "response_b"]] = (pd.DataFrame(df1_normalized[["response_a", "response_b"]].values.tolist()).apply(sorted, axis=1, result_type="expand"))
    df2_normalized[["response_a", "response_b"]] = (pd.DataFrame(df2_normalized[["response_a", "response_b"]].values.tolist()).apply(sorted, axis=1, result_type="expand"))

    df1_normalized["key"] = df1_normalized.apply(lambda row: f"{row['prompt']},{row['response_a']},{row['response_b']}", axis=1)
    df2_normalized["key"] = df2_normalized.apply(lambda row: f"{row['prompt']},{row['response_a']},{row['response_b']}", axis=1)

    common_keys = set(df1_normalized["key"]).intersection(set(df2_normalized["key"]))

    return len(common_keys)

df1 = pd.read_parquet("./argilla-dpo-mix-7k.parquet")
df2 = pd.read_parquet("./mlabonne-orpo-dpo-mix-40k.parquet")
common_count = count_equivalent_rows(df1, df2)
print(f"There are {common_count} rows where all three columns are identical, including cases where `response_a` and `response_b` are interchangeable.")